# Move over project demo
This notebook is the first attempt of applying detection and tracking system for move over projets
- objects are detected and tracked
- objects are visualized in the map
- objects at lanes are counted
- objects that changed lanes are counted


In [ ]:
import cv2
import glob
import os
import sys
import numpy as np

from random import randint
import matplotlib.pyplot as plt

import socket
import json

from time import time
from cav.objects import Object, BoundingBox, ObjectType
from cav.parameters import Parameters
from cav.lanes import Lanes

from cav.visualization import Map, plotBoxes

# Deep sort imports
from deep_sort import nn_matching
from deep_sort.tracker import Tracker
from deep_sort.detection import Detection
%matplotlib inline 

from icecream import ic


## Deep sort

In [ ]:
from helper import ImageEncoder, create_box_encoder, extract_image_patch

In [ ]:
ENCODER_PATH = "./deep_sort_network/mars-small128.pb"
ENCODER_BATCH_SIZE = 32
ENCODER_INPUT_NAME = "images"
ENCODER_OUTPUT_NAME = "features"

image_encoder = ImageEncoder(ENCODER_PATH, ENCODER_INPUT_NAME, ENCODER_OUTPUT_NAME)
encoder = create_box_encoder(ENCODER_PATH, batch_size=32)

## Create tracker

In [ ]:
max_cosine_distance = 0.2
nn_budget = 100

metric = nn_matching.NearestNeighborDistanceMetric(
    "cosine", max_cosine_distance, nn_budget)

### Create object detector

In [ ]:
from cav.detection import ObjectDetector
import cv2
MODEL_PATH = '../../CAV/models/frcnn/inference/saved_model/'
od = ObjectDetector(MODEL_PATH)

### Create params and map objects

In [ ]:
params = Parameters()
params.generateParameters('./config/params_MD295.json')
mymap = Map('./images/satellite_views/MD295.jpg', './config/icons.json', params)
plt.imshow(mymap.getMap(), interpolation='nearest')
plt.show()

### Lane Controller

In [ ]:
from cav.lanes import Lanes
lanes_controller = Lanes('./images/lane_masks/MD295_mask.png')

#### Logging parameters

In [ ]:
PREPARE_VISUALIZATION = True ### Temporary and ugly - Should be cleaned soon

SHOW_TABLE = True

SAVE_LOG = None #### Saves logs with all detected objects (path to file or none)
#SAVE_LOG = 'test_log_20201028.csv'

#### Other parameters

In [ ]:
VIDEO_X = 1440
VIDEO_Y = 880
FRAMES_SEC = 30

MAX_BOXES_TO_DRAW = 100
MIN_SCORE_THRESH = 0.5
IOU_COMMON_THRESHOLD = 0.50
NOT_DETECTED_TRHESHOLD = 1


SKIP_FIRST = 20

MAPSHAPE = mymap.getMap().shape
print ('Y dimension of map is {:.3f} larger than Y dimension of the video'
      .format(MAPSHAPE[0] / VIDEO_Y))

MAP_RESIZE = 3

print ('Y dimension of map is {:.3f} larger than Y dimension of the video. Size of the map is reduced {} times.'
      .format(MAPSHAPE[0] / VIDEO_Y, MAP_RESIZE))


FINAL_X = VIDEO_X + int(MAPSHAPE[1] / MAP_RESIZE)
FINAL_Y = max(VIDEO_Y, int(MAPSHAPE[0] / MAP_RESIZE))

print ('Video size: [{}, {}], Final size: [{}, {}]'
      .format(VIDEO_X, VIDEO_Y, FINAL_X, FINAL_Y))

RESIZE = False


CROP_VID = True
VID_LEFT = 240
VID_RIGHT = 1680
VID_UP = 200


In [ ]:
print (MAPSHAPE[0] / VIDEO_Y)

In [ ]:
lanes_controller = Lanes('./images/lane_masks/MD295_mask.png')

cap = cv2.VideoCapture('/data/MoveOver/Videos/Videos_20210213/MD-295 NB at I-695 - Crash - 02102021@1448 - MDOT SHA CCTV.mp4') 

fourcc = cv2.VideoWriter_fourcc(*'XVID')

prepare_visualization_base = PREPARE_VISUALIZATION

if PREPARE_VISUALIZATION:
    if RESIZE:
        out = cv2.VideoWriter('Test1_output.avi',fourcc, 2*FRAMES_SEC, (int(FINAL_X/2), int(FINAL_Y/2)))
    else:
        out = cv2.VideoWriter('Test1_output.avi',fourcc, 2*FRAMES_SEC, (int(FINAL_X), int(FINAL_Y)))
else:
    out = None


objects = []

results = []
colors = {}


tracker = Tracker(metric)



nr_skipped = 0
i = 0
t = time()

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    while cap.isOpened():
        t2 = time() - t
        sys.stdout.write('{} frames done in {:.1f} seconds ({:.2f} frames/sec)    \r'.format(
            i, t2, i/t2))                   
        #sys.stdout.write("Processing Frame {}     \r".format(i))
        
        
        frame_timeStamp = i/FRAMES_SEC
        
        ret, image = cap.read()

        if nr_skipped < SKIP_FIRST * FRAMES_SEC:
            nr_skipped += 1
            continue
        
        
        if CROP_VID:
            image = image[VID_UP:, VID_LEFT:VID_RIGHT, :]
            
        boxes, scores, classes = od.detect(image, timestamp=frame_timeStamp)
        if len(boxes) >= 1:
            for box in boxes:
                box.updateParams(params)
                
            boxes_array = [[box.xLeft, box.yTop, box.xRight - box.xLeft, box.yBottom - box.yTop] for box in boxes]
            boxes_array = np.array(boxes_array)
            bgr_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
            features = encoder(bgr_image, boxes_array)
            detections = []

            for box, score, objClass, f_vector in zip(boxes, scores, classes, features):
                detection = Detection(
                    [box.xLeft, box.yTop, box.xRight - box.xLeft, box.yBottom - box.yTop], #BBox
                    score, f_vector,
                    objClass
                )
                detection.bbox = box
                detections.append(detection)

            tracker.predict()
            tracker.update(detections)                
            
            # Added to save only important frames DELETE LATER
            PREPARE_VISUALIZATION = True
        else:
            tracker.predict()
            PREPARE_VISUALIZATION = prepare_visualization_base
            
        plotboxes = []
        plotcolors = []
        objects = []

        if len(tracker.tracks) >= 1:
            for track in tracker.tracks:
                if not track.is_confirmed() or track.time_since_update > 1:
                    continue

                if False:
                    bbox = track.to_tlwh()
                    results.append([
                        i, track.track_id, bbox[0], bbox[1], bbox[2], bbox[3]])


                obj = track.trackedObject

                if obj is not None:
                    if obj.color is None:
                        obj.color = (randint(0, 255), randint(0, 255), randint(0, 255))                        
                    plotbox = obj.bboxes[-1]
                    plotbox.trackId = track.track_id
                    plotboxes.append(plotbox)
                    plotcolors.append(obj.color)
                    objects.append(obj)
                    
                    if SHOW_TABLE:
                        lanes_controller.addObject(obj)

            if len(plotboxes) >= 1:
                vid = plotBoxes(image, plotboxes, colors=plotcolors)
            else:
                vid = image.copy()
                mapimg = mymap.getMap()

            if PREPARE_VISUALIZATION:
                mapimg = mymap.addObjects(objects, use_obj_color = True)        
        elif PREPARE_VISUALIZATION:
                vid = image.copy()
                mapimg = mymap.getMap()
                    

        # Adding map
        if PREPARE_VISUALIZATION:
            if MAP_RESIZE != 1: # Reducing the size of the map
                width = int(mapimg.shape[1] / MAP_RESIZE)
                height = int(mapimg.shape[0] / MAP_RESIZE)
                dim = (width, height)
                # resize image
                mapimg = cv2.resize(mapimg, dim, interpolation = cv2.INTER_AREA)                   
                    
            final_image = np.zeros((FINAL_Y, FINAL_X,3),dtype=np.uint8)

            final_image[:VIDEO_Y, :VIDEO_X, :] = vid
            final_image[:mapimg.shape[0], VIDEO_X:VIDEO_X + mapimg.shape[1], :] = mapimg  
            
            if SHOW_TABLE:
                tableimg = lanes_controller.getDataTables(imgsize = (mapimg.shape[1], 300), timeStamp = frame_timeStamp)
                final_image[mapimg.shape[0]:mapimg.shape[0] + tableimg.shape[0], VIDEO_X:VIDEO_X + tableimg.shape[1], :] = tableimg

        if PREPARE_VISUALIZATION & RESIZE: # Resize image
            width = int(final_image.shape[1] / 2)
            height = int(final_image.shape[0] / 2)
            dim = (width, height)
            # resize image
            final_image = cv2.resize(final_image, dim, interpolation = cv2.INTER_AREA)

        if PREPARE_VISUALIZATION:
            cv2.imwrite('./frames/im_{}.jpg'.format(str(i).zfill(5)), final_image)
            if out is not None:
                out.write(final_image)            
                
        if len(objects) > 0:
                                
            if SAVE_LOG is not None:
                logfile = open('./logs/{}'.format(SAVE_LOG, 'w'))
                for obj in objects:
                    line = '{},{},{}'.format(i,time(),obj.getParams(asCsv=true))                               
                    print(line,file=logfile)                    
                                
        i = i+1
        if i > (10+SKIP_FIRST) * FRAMES_SEC:
        #if False:
            break
                  
            
                        
t = time() - t                             
print('\n\n{} frames done in {:.1f} seconds ({:.2f} frames/sec)'.format(
    i, t, i/t))                             
cap.release()
    


# Generate Video

In [ ]:
print ('({}x{}), {:.1f} fps'.format(FINAL_X, FINAL_Y, FRAMES_SEC))

In [ ]:
from helper import generate_video
generate_video(fps = FRAMES_SEC, outputFile='output_video_lanes.avi')